# Chapter 12 내장 자료형 상속과 다중 상속

## 12.1 내장 자료형의 상속은 까다롭다.
`list`나 `dict` 등 내장 자료형은 C 언어로 작성되었기 때문에 사용자가 오버라이드한 코드를 호출하지 않는 경우가 많다.

In [3]:
# 예제 12-1 __setitem__()을 오버라이드한 메서드를 무시하는 dict의 __init__()과 __update__
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
        
dd = DoppelDict(one=1)  # dict 클래스의 __init__은 오버라이드한 __setitem__()을 호출하지 않는다. 
print(dd)

dd['two'] = 2  # [] 연산자는 __setitem__()을 호출하기 때문에 [2, 2]이 value로 저장된다.
print(dd)

dd.update(three=3)  # dict 클래스의 update() 메서드도 오버라이드한 __setitem__() 메서드를 호출하지 않는다.
print(dd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


In [7]:
# 예제 12-2 AnswerDict의 __getitem__()을 지나치는 dict.update()
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
print(ad['a'])  # [] 연산자는 __getitem__()을 호출한다.

d = {}
d.update(ad)  # ad를 이용하여 갱신하는데, __getitem__을 사용해서 갱신하는 것이 아니다.
print(d['a'])

print(d)

42
foo
{'a': 'foo'}


- 해결책: `collections`의 `UserDict`, `UserList`, `UserString` 등을 상속하자.

In [11]:
# 예제 12-3 dict가 아니라 UserDict를 상속하므로 원하는 대로 작동한다
import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1) 
dd['two'] = 2
dd.update(three=3)
print(dd)

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [16]:
class AnswerDict(collections.UserDict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
print(ad['a'])  # [] 연산자는 __getitem__()을 호출한다.

d = {}
d.update(ad)  # ad를 이용하여 갱신하는데, __getitem__을 사용해서 갱신하는 것이 아니다.
print(d['a'])
print(d)


42
42
{'a': 42}
42


<span style='color: red'>분명 `__setitem__()`에 의하여 어딘가에는 'foo'가 저장되어 있을텐데 어디에 있을까? 해시 테이블 안에는 그렇게 저장되어 있겠지? 어떻게 확인할 수 있을까?</span>

In [22]:
for v in ad.values():
    print(v)

42


In [23]:
ad.__dict__

{'data': {'a': 'foo'}}

## 12.2 다중 상속과 메서드 결정 순서
- 다중 상속시 슈퍼 클래스들끼리 서로 같은 이름을 갖는 메서드들이 있다면 어떻게 될까?
- 서브클래스 정의에 나열된 슈퍼클래스의 순서에 따른다. 그리고 이는 `__mro__` 클래스 속성에 명시된다.

In [38]:
# 12-4 diamond.py
class A:
    def ping(self):
        print('ping: ', self)
        
class B(A):
    def pong(self):
        print('pong: ', self)
        
class C(A):
    def pong(self):
        print('PONG', self)
        
class D(B, C):
    def ping(self):
        super().ping()  # A 클래스의 ping 실행
        print('post-ping: ', self)
        
    def pingpong(self):
        self.ping()  # D 클래스의 ping
        super().ping() # A 클래스의 ping
        self.pong()  # B 클래스의 pong
        super().pong()  # B 클래스의 pong
        C.pong(self)  # C 클래스의 pong

In [34]:
d = D()
d.pong()

pong:  <__main__.D object at 0x00000228BAD5B700>


In [35]:
C.pong(d)

PONG <__main__.D object at 0x00000228BAD5B700>


- `__mro__` 클래스 속성은 현재 클래스부터 object 클래스까지 슈퍼클래스들의 MRO (method resolution order, 메서드 결정 순서)를 튜플 형태로 저장

In [36]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [37]:
# 예제 12.6 ping()을 호출하기 위해 super() 사용하기.
d = D()
d.ping()

ping:  <__main__.D object at 0x00000228BAD5B5E0>
post-ping:  <__main__.D object at 0x00000228BAD5B5E0>


In [39]:
# 예제 12.7 pingpong()이 수행한 다섯 번의 호출
d = D()

d.pingpong()

ping:  <__main__.D object at 0x00000228BAD5BA30>
post-ping:  <__main__.D object at 0x00000228BAD5BA30>
ping:  <__main__.D object at 0x00000228BAD5BA30>
pong:  <__main__.D object at 0x00000228BAD5BA30>
pong:  <__main__.D object at 0x00000228BAD5BA30>
PONG <__main__.D object at 0x00000228BAD5BA30>


In [44]:
# 예제 12-8 여러 클래스에서 __mro__속성 정의하기
import io
import numbers
import tkinter

def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))
    
print_mro(bool)
print_mro(numbers.Integral)
print_mro(io.BytesIO)
print_mro(io.TextIOWrapper)
print_mro(tkinter.Text)

bool, int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object
Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## 12.4 다중 상속 다루기
1. 인터페이스 상속과 구현 상속을 구분하라.
   1. 인터페이스 상속: 'is-a' 관계를 의미하는 서브타입을 생성 -> **프레임워크의 중추적 역할**
   2. 구현 상속: 재사용을 통해 코드 중복을 회피 -> **코드 재사용**
2. 인터페이스 상속의 경우 ABC를 이용해서 인터페이스를 명확히 한다.
3. 구현 상속을 위해 믹스인을 사용한다.
   1. 믹스인 클래스는 새로운 자료형을 정의하지 않고, 단지 재사용할 메서드들을 묶어놓을 뿐
   2. 믹스인 클래스를 통해 인스턴스를 생성하며 안 되며, 믹스인 클래스를 상속하는 구상 클래스는 다른 클래스도 상속해야 한다.
   3. 각각의 믹스인 클래스는 밀접히 연관된 메서드 몇 개를 구현해서 하나의 구체적인 행위를 제공해야 한다.
4. 이름을 통해 믹스인임을 명확히 한다
5. ABC가 믹스인이 될 수 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
   1. 믹스인은 자료형을 정의하지 않기 때문에
   2. 믹스인은 반드시 다중 상속을 위해 사용되어야 한다.
6. 두 개 이상의 구상 클래스에서 상속 받지 않는다.
7. 사용자에게 집합 클래스를 제공한다
8. 클래스 상속보다 객체 구성을 사용하라